In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, when
from pyspark.sql.types import StructType, StructField, StringType, IntegerType
from utils.dataloader import DataLoader
from utils.workload import Workload
from utils.cleaner import salvar_csv, salvar_parquet
import sys

# --- Ajuste de path caso necessário ---
sys.path.append("/app")

# --- Inicialização do Spark ---
spark = (
    SparkSession.builder
    .appName("ComparacaoCarregamentoProcessamentoEscrita")
    .master("spark://spark-master:7077")
    .getOrCreate()
)

Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/07/09 14:32:46 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
# --- Caminhos ---
caminho_csv_pessoas     = "/app/full_data/pessoas"
caminho_parquet_pessoas = "/app/full_data/parquet_pessoas"
caminho_saida_parquet   = "/app/full_data/output_faixa_etaria_parquet"
caminho_saida_csv       = "/app/full_data/output_faixa_etaria_csv"

# Schema das pessoas (exemplo parcial — adicione o restante conforme necessário)
schema_pessoas = StructType([
    StructField("id", StringType(), True),
    StructField("pesid", StringType(), True),
    StructField("data_inversa", StringType(), True),
    StructField("dia_semana", StringType(), True),
    StructField("horario", StringType(), True),
    StructField("uf", StringType(), True),
    StructField("br", StringType(), True),
    StructField("km", StringType(), True),
    StructField("municipio", StringType(), True),
    StructField("causa_acidente", StringType(), True),
    StructField("tipo_acidente", StringType(), True),
    StructField("classificacao_acidente", StringType(), True),
    StructField("fase_dia", StringType(), True),
    StructField("sentido_via", StringType(), True),
    StructField("condicao_metereologica", StringType(), True),
    StructField("tipo_pista", StringType(), True),
    StructField("tracado_via", StringType(), True),
    StructField("uso_solo", StringType(), True),
    StructField("id_veiculo", StringType(), True),
    StructField("tipo_veiculo", StringType(), True),
    StructField("marca", StringType(), True),
    StructField("ano_fabricacao_veiculo", StringType(), True),
    StructField("tipo_envolvido", StringType(), True),
    StructField("estado_fisico", StringType(), True),
    StructField("idade", StringType(), True),
    StructField("sexo", StringType(), True),
    StructField("nacionalidade", StringType(), True),
    StructField("naturalidade", StringType(), True)
])

# --- Etapa opcional: converter CSV original para Parquet, se ainda não tiver sido feito ---
print("[ETAPA] Gerando base Parquet a partir do CSV original...")
df_original_csv = DataLoader(spark=spark, base_dir=caminho_csv_pessoas, schema=schema_pessoas).load()
salvar_parquet(df=df_original_csv, output_path=caminho_parquet_pessoas)

[ETAPA] Gerando base Parquet a partir do CSV original...


25/07/09 14:32:50 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.
[Stage 0:>                                                          (0 + 1) / 1]

[PARQUET] salvo em: /app/full_data/parquet_pessoas


DataFrame[id: string, pesid: string, data_inversa: string, dia_semana: string, horario: string, uf: string, br: string, km: string, municipio: string, causa_acidente: string, tipo_acidente: string, classificacao_acidente: string, fase_dia: string, sentido_via: string, condicao_metereologica: string, tipo_pista: string, tracado_via: string, uso_solo: string, id_veiculo: string, tipo_veiculo: string, marca: string, ano_fabricacao_veiculo: string, tipo_envolvido: string, estado_fisico: string, idade: string, sexo: string, nacionalidade: string, naturalidade: string]

In [3]:
# --- Funções de carregamento ---
def load_csv(spark, path):
    return DataLoader(spark=spark, base_dir=path).load()

def load_parquet(spark, path):
    return spark.read.parquet(path)

# --- Função de processamento: Análise por faixa etária ---
def analise_faixa_etaria(df):
    df_limpo = df.withColumn(
        "idade_numerica",
        when(col("idade").rlike("^[0-9]+$"), col("idade").cast(IntegerType()))
        .otherwise(None)
    )

    df_faixa_etaria = df_limpo.withColumn(
        "faixa_etaria",
        when(col("idade_numerica") < 18, "<18")
        .when((col("idade_numerica") >= 18) & (col("idade_numerica") < 30), "18-29")
        .when((col("idade_numerica") >= 30) & (col("idade_numerica") < 45), "30-44")
        .when((col("idade_numerica") >= 45) & (col("idade_numerica") < 60), "45-59")
        .when(col("idade_numerica") >= 60, "60+")
        .otherwise("Desconhecida")
    )

    resultado = df_faixa_etaria.groupBy("faixa_etaria").count().orderBy("faixa_etaria")
    return resultado

# === WORKLOADS ===

# --- WORKLOAD 1: Carregamento ---
df_csv = Workload.run(
    title="1A - Carregamento de dados (CSV)",
    execute_fn=load_csv,
    spark=spark,
    path=caminho_csv_pessoas
)

df_parquet = Workload.run(
    title="1B - Carregamento de dados (Parquet)",
    execute_fn=load_parquet,
    spark=spark,
    path=caminho_parquet_pessoas
)

# --- WORKLOAD 2: Processamento (faixa etária) ---
df_csv_processado = Workload.run(
    title="2A - Processamento faixa etária (CSV)",
    execute_fn=analise_faixa_etaria,
    df=df_csv
)

df_parquet_processado = Workload.run(
    title="2B - Processamento faixa etária (Parquet)",
    execute_fn=analise_faixa_etaria,
    df=df_parquet
)

# --- WORKLOAD 3: Escrita ---
Workload.run(
    title="3A - Escrita resultado faixa etária como Parquet",
    execute_fn=lambda df: salvar_parquet(df=df, output_path=caminho_saida_parquet),
    df=df_parquet_processado
)

Workload.run(
    title="3B - Escrita resultado faixa etária como CSV",
    execute_fn=lambda df: salvar_csv(df=df, output_path=caminho_saida_csv),
    df=df_csv_processado
)

# --- Relatório final com tempos ---
Workload.print_report()


[WORKLOAD] 1A - Carregamento de dados (CSV)


[METRIC] 1A - Carregamento de dados (CSV) executada em 16.369 segundos
[WORKLOAD] 1B - Carregamento de dados (Parquet)
[METRIC] 1B - Carregamento de dados (Parquet) executada em 0.176 segundos
[WORKLOAD] 2A - Processamento faixa etária (CSV)
[METRIC] 2A - Processamento faixa etária (CSV) executada em 0.194 segundos
[WORKLOAD] 2B - Processamento faixa etária (Parquet)
[METRIC] 2B - Processamento faixa etária (Parquet) executada em 0.109 segundos
[WORKLOAD] 3A - Escrita resultado faixa etária como Parquet


[PARQUET] salvo em: /app/full_data/output_faixa_etaria_parquet
[METRIC] 3A - Escrita resultado faixa etária como Parquet executada em 2.483 segundos
[WORKLOAD] 3B - Escrita resultado faixa etária como CSV


[CSV] salvo em: /app/full_data/output_faixa_etaria_csv
[METRIC] 3B - Escrita resultado faixa etária como CSV executada em 12.387 segundos

[WORKLOAD REPORT]
- [WORKLOAD-1] 1A - Carregamento de dados (CSV): 16.369s
- [WORKLOAD-2] 1B - Carregamento de dados (Parquet): 0.176s
- [WORKLOAD-3] 2A - Processamento faixa etária (CSV): 0.194s
- [WORKLOAD-4] 2B - Processamento faixa etária (Parquet): 0.109s
- [WORKLOAD-5] 3A - Escrita resultado faixa etária como Parquet: 2.483s
- [WORKLOAD-6] 3B - Escrita resultado faixa etária como CSV: 12.387s
